In [85]:
import numpy as np
import pandas as pd

import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score


from scipy import stats
from sklearn import preprocessing 
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.scorer import make_scorer
from sklearn.model_selection import GridSearchCV
import statsmodels
from sklearn.compose import ColumnTransformer
import statsmodels.api as sm
from statsmodels.formula.api import ols

import matplotlib.pyplot as plt 
import seaborn as sns
from pandas import Timestamp

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR

import xgboost as xgb
from xgboost import XGBRegressor 
from xgboost.sklearn import XGBClassifier # sklearn’s Grid Search with parallel processing
from xgboost import plot_importance
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from catboost import CatBoostRegressor

# import warnings
# warnings.filterwarnings('ignore')

In [233]:
data = pd.read_csv('DF_File_sample.csv')
data['SKU_Customer'] = data['DemandCustomer'] + data['SKU10']
data.drop(['DemandCustomer', 'SKU10'], axis =1, inplace = True)

### Label Encoding:

In [173]:
cat_var = [data.columns.get_loc(c) for c in data.columns if data.loc[:, c].dtypes=='object']

In [174]:
le = preprocessing.LabelEncoder()

for i in cat_var:
    data.iloc[:, i] = le.fit_transform(data.iloc[:, i])

### Categorical:

In [7]:
for c in cat_var:
    data.iloc[:, c] = pd.Categorical(data.iloc[:,c])

### One Hot Encoder:

In [234]:
data_OH = pd.get_dummies(data)

In [235]:
data = data_OH.copy()

In [94]:
data.dtypes

Year                        int64
Quarter                     int64
Month_No                    int64
Week_No                     int64
Sales                     float64
W_Nielsen                 float64
Brand_0                     uint8
Brand_1                     uint8
Brand_2                     uint8
SKU_Customer_0              uint8
SKU_Customer_1              uint8
SKU_Customer_2              uint8
SKU_Customer_3              uint8
SKU_Customer_4              uint8
SKU_Customer_5              uint8
SKU_Customer_6              uint8
SKU_Customer_7              uint8
SKU_Customer_8              uint8
SKU_Customer_9              uint8
SKU_Customer_10             uint8
SKU_Customer_11             uint8
SKU_Customer_12             uint8
SKU_Customer_13             uint8
SKU_Customer_14             uint8
SKU_Customer_15             uint8
SKU_Customer_16             uint8
SKU_Customer_17             uint8
SKU_Customer_18             uint8
SKU_Customer_19             uint8
SKU_Customer_2

## Data cleaning:

#### agg sales by month

In [236]:
data['year_week'] = data['Year'].astype(str) + '-' + data['Week_No'].astype(str)
data['pre_date'] = data['year_week'].apply(lambda x: datetime.datetime.strptime(x + '-4',  "%G-%V-%w"))

In [237]:
first_null = data.groupby('pre_date').Sales.sum().loc[lambda x: x == 0].sort_values().index[0]
data = data[data.pre_date < first_null]

In [238]:
first_26_week = pd.Series(sorted(data['pre_date'].unique())).iloc[-26]

In [239]:
first_34_week = pd.Series(sorted(data['pre_date'].unique())).iloc[-34]

In [240]:
train = data[data['pre_date'] < first_34_week]
test = data[data['pre_date'] >= first_26_week]
data.drop(['pre_date', 'year_week'], axis =1, inplace = True)
train.drop(['pre_date', 'year_week'], axis =1, inplace = True)
test.drop(['pre_date', 'year_week'], axis =1, inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


#### test preparation:

In [241]:
test['Sales'] = test.groupby(['Year', 'Month_No']).Sales.transform('mean')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [242]:
X_train = train.loc[:, train.columns!='Sales']
X_test = test.loc[:, test.columns!='Sales']
y_train = train['Sales']
y_test = test['Sales']

In [243]:
X_train_val, X_test_val, y_train_val, y_test_val = train_test_split(train.loc[:, train.columns!='Sales'], train['Sales'], test_size=0.33, random_state=42)

In [127]:
def diff_scale(scaling_type = 'norm', X_train = X_train, X_test = X_test):
    norm_scaler = preprocessing.StandardScaler()
    min_max_scaler = preprocessing.MinMaxScaler() # [0, 1]
    max_abs_scaler = preprocessing.MaxAbsScaler() # [-1, 1]
    if scaling_type == 'norm':
        X_train_scaled = norm_scaler.fit_transform(X_train)
        X_test_scaled = norm_scaler.transform(X_test)
    elif scaling_type == 'min_max':
        X_train_scaled = min_max_scaler.fit_transform(X_train)
        X_test_scaled = min_max_scaler.transform(X_test)
    else: 
        X_train_scaled = max_abs_scaler.fit_transform(X_train)
        X_test_scaled = max_abs_scaler.transform(X_test) 
    return X_train_scaled, X_test_scaled

In [244]:
X_train_scaled, X_test_scaled = diff_scale('norm', X_train, X_test)

### scale by column(for categorical variables):

In [15]:
def scale_by_col(df, scaler):
    for var in df.select_dtypes(['number']).columns:
        df[var] = scaler.fit_transform(df[var].values.reshape(-1,1))
    return df

In [294]:
X_train_с_scaled = scale_by_col(X_train_c, norm_scaler)
X_test_c_scaled = scale_by_col(X_test_c, norm_scaler)

### Eval func:

In [245]:
def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return (np.mean(np.abs((y_true - y_pred))) * 100 / np.mean(np.abs((y_true)))) 

In [246]:
def mape_by_month(y_true, y_pred): 
    test2 = test.copy()
    test2['Sales_pred'] = y_pred
    test2['Sales_pred'] = test2.groupby(['Year', 'Month_No']).Sales_pred.transform('mean')
    return mape(test['Sales'], test2['Sales_pred'])

In [251]:
def rmse(y, y_pred):
    return np.sqrt(np.mean(np.square(y - y_pred)))

In [254]:
def rmse_by_month(y_true, y_pred): 
    test2 = test.copy()
    test2['Sales_pred'] = y_pred
    test2['Sales_pred'] = test2.groupby(['Year', 'Month_No']).Sales_pred.transform('mean')
    return rmse(test['Sales'], test2['Sales_pred'])

# Modeling:

## Ridge: 

In [255]:
alphas = np.arange(0, 1, 0.01)
fit_interceptOptions = ([True, False])

model = Ridge() 
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

ridge_cv = GridSearchCV(model, param_grid=dict(alpha=alphas, fit_intercept=fit_interceptOptions), scoring = scorer, n_jobs=-2)
ridge_cv.fit(X_train_scaled, y_train)

print("ridge mape:", mape_by_month(y_test, ridge_cv.predict(X_test_scaled)), ridge_cv.best_params_)
print("ridge rmse:", rmse_by_month(y_test, ridge_cv.predict(X_test_scaled)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


ridge mape: 12.45272728697749 {'alpha': 0.99, 'fit_intercept': True}
ridge rmse: 111.72657100195269


## Lasso:

In [256]:
alphas = np.arange(0, 1, 0.01)
fit_interceptOptions = ([True, False])

model = Lasso() 
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

lasso_cv = GridSearchCV(model, param_grid=dict(alpha=alphas, fit_intercept=fit_interceptOptions), scoring = scorer, n_jobs=-2)
lasso_cv.fit(X_train_scaled, y_train)

print("lasso mape:", mape_by_month(y_test, lasso_cv.predict(X_test_scaled)), lasso_cv.best_params_)
print("lasso rmse:", rmse_by_month(y_test, lasso_cv.predict(X_test_scaled)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


lasso mape: 12.383196692011094 {'alpha': 0.29, 'fit_intercept': True}
lasso rmse: 111.65330647338497


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 19725414.667131424, tolerance: 1416151.2387218664
  positive)


## ElasticNet:

In [257]:
alphas = np.logspace(-5,2,8)
l1_ratio = [.1, .15, .2, .25, .3,.4,.5,.6,.8]

model = ElasticNet() 
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

ElasticNet_cv = GridSearchCV(model, param_grid=dict(alpha=alphas, l1_ratio = l1_ratio, fit_intercept=fit_interceptOptions), scoring = scorer, n_jobs=-2)
ElasticNet_cv.fit(X_train_scaled, y_train)

print("ElasticNet mape:", mape_by_month(y_test, ElasticNet_cv.predict(X_test_scaled)), ElasticNet_cv.best_params_)
print("ElasticNet rmse:", rmse_by_month(y_test, ElasticNet_cv.predict(X_test_scaled)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


ElasticNet mape: 12.291943118682843 {'alpha': 0.01, 'fit_intercept': True, 'l1_ratio': 0.8}
ElasticNet rmse: 112.8411902327515


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 14776034.158742905, tolerance: 1416151.2387218664
  positive)


In [258]:
alphas = [.009, .01, .015, .02,.05,.1]
l1_ratio = [.1,.2,.3,.4,.5,.55, .6, .65,.7,.8]

model = ElasticNet() 
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

ElasticNet_cv = GridSearchCV(model, param_grid=dict(alpha=alphas, l1_ratio = l1_ratio, fit_intercept=fit_interceptOptions), scoring = scorer, n_jobs=-2)
ElasticNet_cv.fit(X_train_scaled, y_train)

print("ElasticNet mape:", mape_by_month(y_test, ElasticNet_cv.predict(X_test_scaled)), ElasticNet_cv.best_params_)
print("ElasticNet rmse:", rmse_by_month(y_test, ElasticNet_cv.predict(X_test_scaled)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


ElasticNet mape: 12.299664157899228 {'alpha': 0.009, 'fit_intercept': True, 'l1_ratio': 0.8}
ElasticNet rmse: 112.71802394398566


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 596096359.1916285, tolerance: 1416151.2387218664
  positive)


#### Lasso takes the best result => tune diff scaling

### tune diff scaling:

##### min_max

In [260]:
X_train_scaled, X_test_scaled = diff_scale('min_max', X_train, X_test)
alphas = np.logspace(-5,2,8)
l1_ratio = [.1, .15, .2, .25, .3,.4,.5,.6,.8]

model = ElasticNet()  
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

ElasticNet_cv_mm = GridSearchCV(model, param_grid=dict(alpha=alphas, l1_ratio=l1_ratio,fit_intercept=fit_interceptOptions), scoring = scorer, n_jobs=-2)
ElasticNet_cv_mm.fit(X_train_scaled, y_train)

print("Lasso mape:", mape_by_month(y_test, ElasticNet_cv_mm.predict(X_test_scaled)), ElasticNet_cv_mm.best_params_)
print("ElasticNet rmse:", rmse_by_month(y_test, ElasticNet_cv_mm.predict(X_test_scaled)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Lasso mape: 50.17694271242768 {'alpha': 10.0, 'fit_intercept': False, 'l1_ratio': 0.8}
ElasticNet rmse: 351.0281888887466


##### max_abs

In [261]:
X_train_scaled, X_test_scaled = diff_scale('max_abs', X_train, X_test)
alphas = np.logspace(-5,2,8)
l1_ratio = [.1, .15, .2, .25, .3,.4,.5,.6,.8]

model = ElasticNet()  
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

ElasticNet_cv_ma = GridSearchCV(model, param_grid=dict(alpha=alphas, l1_ratio=l1_ratio,fit_intercept=fit_interceptOptions), scoring = scorer, n_jobs=-2)
ElasticNet_cv_ma.fit(X_train_scaled, y_train)

print("Lasso mape:", mape_by_month(y_test, ElasticNet_cv_ma.predict(X_test_scaled)), ElasticNet_cv_mm.best_params_)
print("ElasticNet rmse:", rmse_by_month(y_test, ElasticNet_cv_ma.predict(X_test_scaled)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Lasso mape: 62.50765658487119 {'alpha': 10.0, 'fit_intercept': False, 'l1_ratio': 0.8}
ElasticNet rmse: 426.73585900075085


# SVR:

In [264]:
X_train_scaled, X_test_scaled = diff_scale('norm', X_train, X_test)
parameters = {'kernel': ('linear', 'rbf','poly'),
              'C':[1.5, 10],
              'gamma': [1e-7, 1e-4],
              'epsilon':[0.1,0.2,0.5,0.3]}

model = SVR()
svr = GridSearchCV(model, parameters, scoring = scorer, n_jobs=-2)
svr.fit(X_train_scaled, y_train)

print("SVR mape:", mape_by_month(y_test, svr.predict(X_test_scaled)), svr.best_params_)
print("SVR rmse:", rmse_by_month(y_test, svr.predict(X_test_scaled)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


KeyboardInterrupt: 

### How to predict?)

In [162]:
test2 = test.copy()
test2['Sales_pred'] = ElasticNet_cv_mm1.predict(X_test_scaled)
test2['Sales_pred'] = test2.groupby(['Year', 'Month_No']).Sales_pred.transform('mean')
mape(test['Sales'], test2['Sales_pred'])

In [164]:
mape(test['Sales'], test2['Sales_pred'])

13.884642663194404

In [165]:
train2 = test.copy()
train2['Sales'] = train2.groupby(['Year', 'Month_No']).Sales.transform('mean')

In [166]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(train2.loc[:, train2.columns!='Sales'], train2['Sales'], test_size=0.33, random_state=42)

In [169]:
X_train_scaled, X_test_scaled = diff_scale('min_max', X_train2, X_test2)
alphas = np.logspace(-5,2,8)
l1_ratio = [.1, .15, .2, .25, .3,.4,.5,.6,.8]

model = ElasticNet() 
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

ElasticNet_cv_mm1 = GridSearchCV(model, param_grid=dict(alpha=alphas, l1_ratio = l1_ratio, fit_intercept=fit_interceptOptions), scoring = scorer, n_jobs=-2)
ElasticNet_cv_mm1.fit(X_train_scaled, y_train2)

print("ElasticNet mape:", mape(y_test2, ElasticNet_cv_mm1.predict(X_test_scaled)), ElasticNet_cv_mm1.best_params_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


ElasticNet mape: 7.199229566291241 {'alpha': 1e-05, 'fit_intercept': True, 'l1_ratio': 0.8}


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1187397.4431065638, tolerance: 936.1200684948726
  positive)


In [171]:
mape(test['Sales'],  ElasticNet_cv_mm1.predict(X_test_scaled))

6.613945216159089

### ...

## Lightgbm:

In [268]:
X_train_scaled, X_test_scaled = diff_scale('norm', X_train, X_test)
param_grid = {
    'n_estimators' : list(range(100, 180,10)),
    'num_leaves': list(range(7,10, 1)),
    # 'min_data_in_leaf': [10, 20, 40, 60, 100],
    'max_depth': [2, 4, 6, 7, 8],
    'learning_rate': [0.01, .016, .02, .1]}
    # 'bagging_freq': [3, 4, 5, 6, 7],
    # 'bagging_fraction': np.linspace(0.6, 0.95, 10),
    # 'reg_alpha': np.linspace(0.1, 0.95, 10),
    # 'reg_lambda': np.linspace(0.1, 0.95, 10)
                                                  
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better = False)   
gbm = GridSearchCV(lgb.LGBMRegressor(cat_features= cat_var), 
                   param_grid, 
                   scoring = scorer, n_jobs = -2)


gbm.fit(X_train_scaled, y_train)

print("lightgbm mape:", mape_by_month(y_test, gbm.predict(X_test_scaled)), gbm.best_params_)
print("lightgbm rmse:", rmse(y_test, gbm.predict(X_test_scaled)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


lightgbm mape: 12.344509310978669 {'learning_rate': 0.02, 'max_depth': 7, 'n_estimators': 160, 'num_leaves': 9}
lightgbm rmse: 897.7053086277743


In [272]:
X_train_scaled, X_test_scaled = diff_scale('norm', X_train, X_test)
param_grid = {
    'n_estimators' : list(range(120, 180,10)),
    'num_leaves': list(range(8,10, 1)),
    # 'min_data_in_leaf': [10, 20, 40, 60, 100],
    'max_depth': [6, 7, 8],
    'learning_rate': [ .02, .03, .04,.05]}
    # 'bagging_freq': [3, 4, 5, 6, 7],
    # 'bagging_fraction': np.linspace(0.6, 0.95, 10),
    # 'reg_alpha': np.linspace(0.1, 0.95, 10),
    # 'reg_lambda': np.linspace(0.1, 0.95, 10)
                                                  
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better = False)   
gbm = GridSearchCV(lgb.LGBMRegressor(cat_features= cat_var), 
                   param_grid, 
                   scoring = scorer, n_jobs = -2)


gbm.fit(X_train_scaled, y_train)

print("lightgbm mape:", mape_by_month(y_test, gbm.predict(X_test_scaled)), gbm.best_params_)
print("lightgbm rmse:", rmse(y_test, gbm.predict(X_test_scaled)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


KeyboardInterrupt: 

### Tunning other parameters:

In [271]:
param_grid = {
    'n_estimators' : [160],
    'num_leaves': [9],
    'min_data_in_leaf': [9, 10, 11, 12, 13],
    'max_depth': [7],
    'learning_rate': [.2],
    # 'bagging_freq': [3, 4, 5, 6, 7],
    # 'bagging_fraction': np.linspace(0.6, 0.95, 10),
    'reg_alpha': np.linspace(0.1, 1, 10),
    'reg_lambda': np.linspace(0.1, 1, 10)}
                                                  
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better = False)   
gbm2 = GridSearchCV(lgb.LGBMRegressor(cat_features= cat_var), 
                   param_grid, 
                   scoring = scorer, n_jobs = -2)


gbm2.fit(X_train_scaled, y_train)

print("lightgbm mape:", mape_by_month(y_test, gbm.predict(X_test_scaled)), gbm.best_params_)
print("lightgbm rmse:", rmse(y_test, gbm.predict(X_test_scaled)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


lightgbm mape: 14.938325563186188 {'learning_rate': 0.2, 'max_depth': 7, 'min_data_in_leaf': 9, 'n_estimators': 160, 'num_leaves': 9, 'reg_alpha': 0.6, 'reg_lambda': 0.5}
lightgbm rmse: 1073.7605941069442


# XGB

In [274]:
param_grid = {
    'n_estimators' : list(range(150, 400, 50)),
    # 'num_leaves': list(range(8, 16, 4)),
    # 'min_data_in_leaf': [10, 20, 40, 60, 100],
    'max_depth': [8, 12, 15],
    'learning_rate': [.3, 0.1, 0.01]}
    # 'bagging_freq': [3, 4, 5, 6, 7],
    # 'bagging_fraction': np.linspace(0.6, 0.95, 10),
    # 'reg_alpha': np.linspace(0.1, 0.95, 10),
    # 'reg_lambda': np.linspace(0.1, 0.95, 10)
                          
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better = False) 
model_xgb = xgb.XGBRegressor(objective = 'reg:squarederror')                        

    
grid_xgb = GridSearchCV(model_xgb, 
                        param_grid, 
                        scoring = scorer,
                        n_jobs=-2,
                        verbose=2)

grid_xgb.fit(X_train_scaled, y_train)

[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 45 candidates, totalling 135 fits


[Parallel(n_jobs=-2)]: Done  35 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-2)]: Done 135 out of 135 | elapsed: 23.1min finished
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:squarederror',
                                    rand..., reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='warn', n_jobs=-2,
             param_grid={'learning_rate': [0.3, 0.1, 0.01],
                         'max_depth': [8, 12, 15],
                      

In [275]:
print("XGB mape:", mape_by_month(y_test, grid_xgb.predict(X_test_scaled)), grid_xgb.best_params_)
print("XGB rmse:", rmse(y_test, grid_xgb.predict(X_test_scaled)))

XGB mape: 15.124508421639051 {'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 250}
XGB rmse: 1049.433300771891


In [276]:
param_grid = {
    'n_estimators' : list(range(200, 300, 50)),
    # 'num_leaves': list(range(8, 16, 4)),
    # 'min_data_in_leaf': [10, 20, 40, 60, 100],
    'max_depth': [6,7,8],
    'learning_rate': [.005, .009, .01, 0.2]}
    # 'bagging_freq': [3, 4, 5, 6, 7],
    # 'bagging_fraction': np.linspace(0.6, 0.95, 10),
    # 'reg_alpha': np.linspace(0.1, 0.95, 10),
    # 'reg_lambda': np.linspace(0.1, 0.95, 10)
                          
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better = False) 
model_xgb = xgb.XGBRegressor(objective = 'reg:squarederror')                        

    
grid_xgb = GridSearchCV(model_xgb, 
                        param_grid, 
                        scoring = scorer,
                        n_jobs=-2,
                        verbose=2)

grid_xgb.fit(X_train_scaled, y_train)

print("XGB mape:", mape_by_month(y_test, grid_xgb.predict(X_test_scaled)), grid_xgb.best_params_)
print("XGB rmse:", rmse(y_test, grid_xgb.predict(X_test_scaled)))

Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  35 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-2)]: Done  72 out of  72 | elapsed:  7.1min finished


XGB mape: 19.60834774181943 {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 200}
XGB rmse: 938.1413436965049
